In [1]:
import sys
sys.path.append('/Users/dan/Code/Python/pg_scaffold/generated')

In [2]:

from app.core.db import SessionLocal

db = SessionLocal()
skip = 0
limit = 100

In [3]:

from typing import List
from passlib.context import CryptContext
from fastapi import APIRouter, Depends, HTTPException, Query
from sqlalchemy.orm import Session

from app.crud.user import CRUDUser
from app.schemas import user as user_schema

pwd_context = CryptContext(schemes=["argon2"], deprecated="auto")

In [4]:
#Get user by email

user_json = {"email": "dphiggins3@gmail.com"}

errors = {}
filters = {"email":user_json["email"]}
service = CRUDUser(db)
user_objs = service.get_many_like(filters)

# If we found an email check the password
if user_objs:
    user_obj = user_objs[0]
    print(user_objs[0])

id=3 first_name='tom' last_name='foley' email='dphiggins3@gmail.com' password='$argon2id$v=19$m=65536,t=3,p=4$WSuldE7JOed8b23N2RvjHA$4N7zzKJ9zVcKZOsA10Swx1E1STo7bBDPKguAks5a6Ag' organization=None magic_link_token=None magic_link_expires_at=None last_login_at=None is_verified=False registered_at=datetime.datetime(2025, 10, 23, 12, 7, 24, 83742) role='user'


In [ ]:
#Get user by id

errors = {}
service = CRUDUser(db)
user_obj = service.get_by_id(id=3)
print(user_obj)

In [ ]:
#Get all users

errors = {}
service = CRUDUser(db)
user_objs = service.get_many()

# If we found an email check the password
if user_objs:
    user_obj = user_objs[0]
    print(user_objs[0])

In [ ]:
# Create a new user

user_json = {"first_name":"tom",
            "last_name": "foley",
            "email": "dphiggins4@gmail.com",
            "password": "Password"}


user_json["password"] = pwd_context.hash(user_json["password"])

db_obj = None
try:
    user_in = user_schema.UserCreate.model_validate(user_json)
    service = CRUDUser(db)
    db_obj = service.create(obj_in=user_in)
except Exception as e:
        db.rollback()  # Rollback the transaction in case of error
        raise HTTPException(status_code=500, detail=str(e))

print(db_obj)

In [9]:
#Get user by id

errors = {}
service = CRUDUser(db)
user_obj = service.get_by_id(id=5)
print(user_obj)

ValidationError: 1 validation error for UserRead
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/model_attributes_type

In [6]:
#Update user
user_json = {"id":5,
             "first_name":"tommy",
            "last_name": "foley",
            "email": "dphiggins4@gmail.com",
            "password": "Password"}
service = CRUDUser(db)
user_obj = service.update(obj_in=user_schema.UserUpdate.model_validate(user_json))
print(user_obj)


id=5 first_name='tommy' last_name='foley' email='dphiggins4@gmail.com' password='Password' organization=None magic_link_token=None magic_link_expires_at=None last_login_at=None is_verified=False registered_at=datetime.datetime(2025, 10, 23, 14, 50, 21, 683097) role='user'


In [8]:
#Delete user

service = CRUDUser(db)
user_obj = service.remove(id=5)
print(user_obj)

id=5 first_name='tommy' last_name='foley' email='dphiggins4@gmail.com' password='Password' organization=None magic_link_token=None magic_link_expires_at=None last_login_at=None is_verified=False registered_at=datetime.datetime(2025, 10, 23, 14, 50, 21, 683097) role='user'
